In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import time
from sklearn.preprocessing import MinMaxScaler

# Models

* FM

In [ ]:
def initialization(x, mean=0., std=1.):
    return x.normal_().fmod_(2).mul_(std).add_(mean)

In [ ]:
class FM(nn.Module):
    def __init__(self, n, k):
        super().__init__()
        self.bias = nn.Embedding(n, 1)
        self.embeddings = nn.Embedding(n, k)

        with torch.no_grad(): 
            initialization(self.embeddings.weight, std=0.01)
            initialization(self.bias.weight, std=0.01)

    def forward(self, X):

        dense_emb = self.embeddings(X)

        p1 = dense_emb.sum(dim=1).pow(2)
        p2 = dense_emb.pow(2).sum(dim=1)

        interaction_layer = 0.5 * (p1-p2).sum(1)
        linear_layer = self.bias(X).squeeze().sum(1)

        return linear_layer + interaction_layer


* BPR Loss

In [ ]:
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

class BPRLoss(nn.Module):
    def __init__(self, item1 = None, item2 = None):
        super(BPRLoss, self).__init__()
        self.item1 = item1
        self.item2 = item2

    def forward(self, item1, item2):
        dist = item1 - item2
        return -torch.sum(torch.log(sigmoid(dist)))


# Train

In [ ]:
class MovieLens(Dataset):
    def __init__(self, pair1, pair2):
        self.pair1 = pair1
        self.pair2 = pair2

    def __len__(self):
        return len(self.pair1)

    def __getitem__(self, idx):
        p1 = self.pair1[idx]
        p2 = self.pair2[idx]
        return p1, p2


def data_tensor(dataset):
    scaler = MinMaxScaler()

    user_feature = dataset.iloc[:,0:3]
    movie1_feature = dataset.iloc[:,3:5]
    movie2_feature = dataset.iloc[:,5:7]
    
    positive = user_feature.join(movie1_feature)
    negative = user_feature.join(movie2_feature)

    # positive = scaler.fit_transform(positive.values)
    # positive = torch.tensor(positive).int()

    # negative = scaler.fit_transform(negative.values)
    # negative = torch.tensor(negative).int()
    positive = torch.tensor(positive.values).int()
    negative = torch.tensor(negative.values).int()

    return MovieLens(positive, negative)


In [ ]:
batch_size = 1024

train_set_path = {"full": '',
                "sampled":"./data/revised_bpr_sampled_train_set.csv" }
val_set_path = {"full": '',
                "sampled":"./data/revised_bpr_sampled_val_set.csv" }


train_set = pd.read_csv(train_set_path['sampled'], header=0)
val_set = pd.read_csv(val_set_path['sampled'], header=0)


train_set = data_tensor(train_set)
val_set = data_tensor(val_set)


train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(model, dataloader, optimizer, criterion):
    train_loss = 0
    model.train()
    for pair1, pair2 in dataloader:
        pair1, pair2 = pair1.to(device), pair2.to(device)
        optimizer.zero_grad()
        score1, score2 = model(pair1), model(pair2)
        loss = criterion(score1, score2)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    return train_loss / len(dataloader.dataset)


In [ ]:
def validation(model, dataloader, criterion):
    val_loss = 0
    model.eval()
    for pair1, pair2 in dataloader:
        pair1, pair2 = pair1.to(device), pair2.to(device)
        with torch.no_grad():
            score1, score2 = model(pair1), model(pair2)
        loss = criterion(score1, score2)
        val_loss += loss.item()

    return val_loss / len(dataloader.dataset)

In [ ]:
num_features = 10052
num_dimention = 100
model = FM(num_features,num_dimention)
wd=1e-5
lr=1e-4
epochs=20

optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[4], gamma=0.1)
criterion = BPRLoss()

train_loss_list = []
val_loss_list = []

for epoch in range(epochs):

    train_loss = train(model, train_dataloader, optimizer, criterion)
    val_loss = validation(model, val_dataloader, criterion)
    scheduler.step()

    sqrt_train_loss = (math.sqrt(train_loss))
    sqrt_val_loss = (math.sqrt(val_loss))
    
    print(f'epoch {epoch}:')
    print(f'\ttrain loss: {train_loss:.4f}')
    print(f'\tvalidation loss: {val_loss:.4f}')
    if epoch % 5 == 0:
      file_name = './model/model_'+str(epoch)+'.pth'
      torch.save(model.state_dict(), file_name)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)

In [ ]:

#plot 1:
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs+1), train_loss_list)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training')

#plot 2:
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs+1), val_loss_list)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Validation')

plt.show()
# plt.savefig('image.png')

In [ ]:
torch.save(model.state_dict(), './model/model_v6.pth')